- Проверить наличие в справочниках адресов, брендов, граммовок, регионов (если нет добавить)
- Взять файл
- Заменить адреса, субъекты, бренды, вес на корректные
- Объединить по месяцу, году, региону, бренду, адресу, сумма по весу
- то же по конкурентам
- объединить их вместе
- выгрузить в файл
- убрать адрес Санкт Петербург, Пироговская наб., д.17
- сделать сводный

In [193]:
import pandas as pd

from datetime import datetime

import xlrd

### импорт нужных таблиц

In [194]:
path = 'D:\\Analysis Burn'

In [195]:
burn = pd.read_csv(path + '\\Википедия\\БЁРН исходники\\burn_tobacco.csv')
dsmh = pd.read_csv(path + '\\Википедия\\СВМХ исходники\\dsmh.csv')

In [196]:
addresses = pd.read_excel(path + '\\Википедия\\Справочники\\addresses.xlsx')

In [197]:
regions = pd.read_excel(path + '\\Википедия\\Справочники\\regions.xlsx')

In [198]:
brands = pd.read_excel(path + '\\Википедия\\Справочники\\brands.xlsx')

In [199]:
weight = pd.read_excel(path + '\\Википедия\\Справочники\\weight.xlsx')

In [200]:
burn_products_price = pd.read_excel(r"D:\Analysis Burn\Википедия\Справочники\burn_products_price.xlsx")

In [201]:
dsmh_products_price = pd.read_excel(r"D:\Analysis Burn\Википедия\Справочники\dsmh_products_price.xlsx")

# Получение отсутствующих значений в справочниках

In [202]:
addresses_bad = pd.concat([dsmh['адрес'], burn['Адрес']] )

In [203]:
addresses_bad = addresses_bad.drop_duplicates()
addresses_bad = addresses_bad.dropna()

In [204]:
addresses_bad = pd.DataFrame(addresses_bad)

In [205]:
addresses_bad = addresses_bad.rename(columns={0:'Адрес'})

In [206]:
merged = pd.merge(addresses, addresses_bad, left_on='Исходный адрес', right_on = 'Адрес', how='outer', indicator=True)

In [207]:
missing_values = merged.loc[ merged['_merge'] == 'right_only' ]

In [208]:
if missing_values.empty == False:
    missing_values.to_excel(r'C:\Users\User\Desktop\missing_addresses.xlsx', index=False)
else:
    print("Он пустой")

Он пустой


In [209]:
regions_bad = pd.concat([dsmh['рег'], burn['Субъект РФ']] )

In [210]:
regions_bad = regions_bad.drop_duplicates()
regions_bad = regions_bad.dropna()

In [211]:
regions_bad = pd.DataFrame(regions_bad)

In [212]:
regions_bad =regions_bad.rename(columns={0:'Регион'})

In [213]:
merged2 = pd.merge(regions, regions_bad, left_on='Субъект РФ исходный', right_on = 'Регион', how='outer', indicator=True)

In [214]:
missing_values2 = merged2.loc[merged2['_merge'] == 'right_only', 'Регион']

In [215]:
if missing_values2.empty == False:
    missing_values2.to_excel(r'C:\Users\User\Desktop\missing_regions.xlsx')
else:
    print("Он пустой")

In [216]:
brands_bad = pd.concat([dsmh['бренд'], burn['Наименование товара']] )

In [217]:
brands_bad = brands_bad.drop_duplicates()
brands_bad = brands_bad.dropna()

In [218]:
brands_bad = pd.DataFrame(brands_bad)

In [219]:
brands_bad = brands_bad.rename(columns={0:'Бренд'})

In [220]:
merged3 = pd.merge(brands, brands_bad, left_on='Бренд (исходный)', right_on = 'Бренд', how='outer', indicator=True)

In [221]:
missing_values3 = merged3.loc[merged3['_merge'] == 'right_only', 'Бренд_y']

In [222]:
if missing_values3.empty == False:
    missing_values3.to_excel(r'C:\Users\User\Desktop\missing_brands.xlsx')
else:
    print("Он пустой")

Он пустой


In [223]:
weight_bad = pd.concat([dsmh['вес'], burn['Наименование товара']] )

In [224]:
weight_bad = weight_bad.drop_duplicates()

In [225]:
weight_bad = pd.DataFrame(weight_bad)

In [226]:
weight_bad = weight_bad.rename(columns={0:'Граммовка'})

In [227]:
merged4 = pd.merge(weight, weight_bad, left_on='Исходная граммовка', right_on = 'Граммовка', how='outer', indicator=True)

In [228]:
missing_values4 = merged4.loc[merged4['_merge'] == 'right_only', 'Граммовка_y']

In [229]:
if missing_values4.empty == False:
    missing_values4.to_excel(r'C:\Users\User\Desktop\missing_weight.xlsx')
else:
    print("Он пустой")

Он пустой


### Проверка наличия цен

In [230]:
dsmh_products_price['Наименование товара'] = dsmh_products_price['продукт']+dsmh_products_price['вес'].astype(str)
dsmh_products_price = dsmh_products_price[['продукт','Наименование товара', 'Цена']]

In [231]:
burn_products_price['продукт'] = burn_products_price['Наименование товара']
burn_products_price = burn_products_price[['продукт', 'Наименование товара','Цена']]

In [232]:
prices = pd.concat([dsmh_products_price, burn_products_price] )

In [233]:
prices = prices.drop_duplicates()
prices = prices.dropna()

In [234]:
prices = pd.DataFrame(prices)

In [235]:
burn_product = pd.DataFrame()
burn_product['Наименование товара'] = burn['Наименование товара']

In [236]:
dsmh_product = pd.DataFrame()
dsmh_product['Наименование товара'] = dsmh['продукт']+dsmh['вес'].astype(str)

In [237]:
products_for_check = pd.concat([burn_product, dsmh_product])

In [238]:
merged = pd.merge(prices, products_for_check, on='Наименование товара', how='outer', indicator=True)

In [239]:
missing_values5 = merged.loc[ merged['_merge'] == 'right_only' ]

In [240]:
if missing_values5.empty == False:
    missing_values5.to_excel(r'C:\Users\User\Desktop\missing_prices.xlsx', index=False)
else:
    # здесь если не пусто, то удалять в конце цифры
    print("Он пустой")
    

Он пустой


# РАБОТА С ФАЙЛОМ BURN

### адреса из справочника

In [241]:
need_addresses = addresses[['Исходный адрес', 'Адрес']]

In [242]:
burn_good_address = burn.merge(need_addresses, how="left", left_on='Адрес', right_on='Исходный адрес')

In [243]:
burn_good_address = burn_good_address.drop(columns=['Исходный адрес', 'Адрес_x'])

In [244]:
burn_good_address = burn_good_address.rename(columns={"Адрес_y": "Адрес"})

### регионы(субъекты) из справочника

In [245]:
burn_good_address = burn_good_address.merge(
    regions, how="left", left_on="Субъект РФ", right_on="Субъект РФ исходный", suffixes=('_left', '_right')
)

In [246]:
burn_good_address = burn_good_address.drop(columns=['Субъект РФ исходный', 'Субъект РФ_left'])

In [247]:
burn_good_address = burn_good_address.rename(columns={"Субъект РФ_right": "Субъект РФ"})

### изменить тип на дату и вычленить дату

In [248]:
burn_good_address['Дата операции'] = burn_good_address['Дата операции'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d')
)

In [249]:
burn_good_address['Месяц'] = burn_good_address['Дата операции'].dt.month

In [250]:
burn_good_address['Год'] = burn_good_address['Дата операции'].dt.year

### заменить бренды из справочника

In [251]:
burn_good_address = burn_good_address.merge(
    brands, how="left", left_on="Наименование товара", right_on="Бренд (исходный)", suffixes=('_left', '_right')
)

In [252]:
burn_good_address = burn_good_address.drop(columns=['Бренд (исходный)'])

### заменить граммовку

In [253]:
burn_good_address = burn_good_address.merge(
    weight, how="left", left_on="Наименование товара", right_on="Исходная граммовка", suffixes=('_left', '_right')
)

In [254]:
burn_good_address = burn_good_address.drop(columns=['Исходная граммовка'])

### Добавление колонки вес

In [255]:
burn_good_address['Граммовка'] = burn_good_address['Граммовка'].astype(int)

In [256]:
burn_good_address['Вес'] = (burn_good_address['Граммовка']/1000)* burn_good_address['Объем товара (пачек)']

In [257]:
burn_good_address['Адрес'] = burn_good_address['Адрес'].fillna('не указан')

### цены из справочника

In [258]:
burn_good_address = burn_good_address.merge(prices, how="left", on='Наименование товара')

In [259]:
# должен быть ноль
burn_good_address['Цена'].isna().sum()

0

In [260]:
burn_good_address['Цена'] = burn_good_address['Цена'] * burn_good_address['Объем товара (пачек)']

### Группировка

In [261]:
burn_good = burn_good_address.groupby(['Год', 'Месяц', 'Субъект РФ', 'Бренд', 'Адрес']).agg({'Вес':'sum','Цена':'sum'}).reset_index()

In [262]:
burn_good.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241234 entries, 0 to 241233
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Год         241234 non-null  int64  
 1   Месяц       241234 non-null  int64  
 2   Субъект РФ  241234 non-null  object 
 3   Бренд       241234 non-null  object 
 4   Адрес       241234 non-null  object 
 5   Вес         241234 non-null  float64
 6   Цена        241234 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 12.9+ MB


# РАБОТА С ФАЙЛОМ MHDS

In [263]:
dsmh_copy = dsmh.copy(deep = True)

### Выделить месяц и год

In [264]:
dsmh_copy['Дата операции'] = dsmh_copy['мес'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d')
)

In [265]:
dsmh_copy['Месяц'] = dsmh_copy['Дата операции'].dt.month

In [266]:
dsmh_copy['Год'] = dsmh_copy['Дата операции'].dt.year

### регион из справочника

In [267]:
dsmh_copy = dsmh_copy.merge(
    regions, how="left", left_on="рег", right_on="Субъект РФ исходный", suffixes=('_left', '_right')
)

In [268]:
dsmh_copy = dsmh_copy.drop(columns=['Субъект РФ исходный', 'рег'])

### адрес из справочника

In [269]:
need_addresses = addresses[['Исходный адрес', 'Адрес']]

In [270]:
dsmh_good_address = dsmh_copy.merge(need_addresses, how="left", left_on='адрес', right_on='Исходный адрес')

In [271]:
dsmh_good_address = dsmh_good_address.drop(columns=['Исходный адрес', 'адрес'])

### цены из справочника

In [272]:
dsmh_good_address['Наименование товара'] = dsmh_good_address['продукт'] + dsmh_good_address['вес'].astype(str)

In [273]:
dsmh_good_address = dsmh_good_address.merge(prices, how="left", on='Наименование товара')

In [274]:
dsmh_good_address['Цена'].isna().sum()

0

In [275]:
dsmh_good_address['Цена'] = dsmh_good_address['Цена'] * dsmh_good_address['колво']

### вес из кол-во и граммовки (у них вес называется)

In [276]:
dsmh_good_address['Новый вес'] = (dsmh_good_address['вес']/1000) * dsmh_good_address['колво']

In [277]:
dsmh_good_address = dsmh_good_address.drop(columns=['вес', 'колво'])

In [278]:
dsmh_good_address = dsmh_good_address.rename(columns = {'Новый вес': 'Вес'})

### бренд из справочника (менять не надо)

In [279]:
dsmh_good_address = dsmh_good_address.rename(columns={'бренд':'Бренд'})

### Группировка

In [280]:
dsmh_good_address = dsmh_good_address.groupby(['Год', 'Месяц', 'Субъект РФ', 'Бренд', 'Адрес']).agg({'Вес':'sum','Цена':'sum'}).reset_index()

In [281]:
dsmh_good_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369284 entries, 0 to 369283
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Год         369284 non-null  int64  
 1   Месяц       369284 non-null  int64  
 2   Субъект РФ  369284 non-null  object 
 3   Бренд       369284 non-null  object 
 4   Адрес       369284 non-null  object 
 5   Вес         369284 non-null  float64
 6   Цена        369284 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 19.7+ MB


In [282]:
(dsmh_good_address.columns == burn_good.columns).all()

True

In [288]:
BBDSMH = pd.concat([dsmh_good_address, burn_good], ignore_index=True)

In [289]:
# удалить ненужный адрес
BBDSMH = BBDSMH.loc[ BBDSMH['Адрес'] != 'г Санкт-Петербург, наб Пироговская, д 17 к 7' ]
BBDSMH = BBDSMH.loc[ BBDSMH['Адрес'] != 'г Санкт-Петербург, наб Пироговская, д 17 к 7 литера а' ]


In [290]:
BBDSMH = BBDSMH.loc[ (BBDSMH['Бренд'] == 'DARKSIDE') | (BBDSMH['Бренд'] == 'MUST HAVE') | (BBDSMH['Бренд'] == 'Blackburn') ]

In [8]:
BBDSMH.loc[BBDSMH['Бренд'] == 'DARKSIDE'] = "darkside"
BBDSMH.loc[BBDSMH['Бренд'] == 'MUST HAVE'] = "must have"
BBDSMH.loc[BBDSMH['Бренд'] == 'Blackburn'] = "BLACKBURN"

In [291]:
BBDSMH.to_excel(r'D:\Analysis Burn\Википедия\Wikipedia.xlsx', index=False)